In [69]:
from itertools import islice
with open("wikicorpus_01") as myfile:
    head = list(islice(myfile, 5651))

In [70]:
print(head[:9])

['Néstor_Gabriel_Martinena néstor_gabriel_martinena NP00000 0\n', '( ( Fpa 0\n', 'La el DA0FS0 0\n', 'Plata plata NP00000 10501473\n', ', , Fc 0\n', 'Buenos_Aires buenos_aires NP00000 0\n', ', , Fc 0\n', 'Argentina argentina NP00000 02035234\n', ', , Fc 0\n']


In [71]:
sent = []
for s in head:
    sent.append(s.split())

In [5]:
sent[:5]

[['Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'],
 ['(', '(', 'Fpa', '0'],
 ['La', 'el', 'DA0FS0', '0'],
 ['Plata', 'plata', 'NP00000', '10501473'],
 [',', ',', 'Fc', '0']]

In [72]:
words = []
sentences = []
for s in sent:
    if len(s) == 4 and s[0] not in {'(', ')', ','} and '[' not in s[1]:
        words.append((s[1], s[2], s[3]))
        #if 'endofarticle' in s[1]:
        #    sentences.append(words)
        #    words = []

In [73]:
count = dict()
for fst,snd,trd in words:
    if fst not in count:
        count[fst] = 1
    else:
        count[fst] += 1

In [74]:
count['entrar']

2

In [40]:
words[:4]

[('néstor_gabriel_martinena', 'NP00000', '0'),
 ('el', 'DA0FS0', '0'),
 ('plata', 'NP00000', '10501473'),
 ('buenos_aires', 'NP00000', '0')]

In [75]:
aux = []
check = []
for fst,snd,trd in words:
    aux.append((fst,snd, trd))
    if fst == '.':
        check.append(aux)
        aux = []

In [137]:
check

[[('néstor_gabriel_martinena', 'NP00000', '0'),
  ('el', 'DA0FS0', '0'),
  ('plata', 'NP00000', '10501473'),
  ('buenos_aires', 'NP00000', '0'),
  ('argentina', 'NP00000', '02035234'),
  ('ser', 'VSIP3S0', '01775973'),
  ('uno', 'DI0MS0', '0'),
  ('futbolista', 'NCCS000', '0'),
  ('argentino', 'AQ0MS0', '00278090'),
  ('.', 'Fp', '0')],
 [('jugar', 'VMM02S0', '00727813'),
  ('de', 'SPS00', '0'),
  ('delantero', 'NCMS000', '00466114'),
  ('y', 'CC', '0'),
  ('su', 'DP3CS0', '0'),
  ('equipo', 'NCMS000', '06093198'),
  ('actual', 'AQ0CS0', '01667781'),
  ('ser', 'VSIP3S0', '01775973'),
  ('gimnasia', 'NP00000', '00275488'),
  ('y', 'CC', '0'),
  ('esgrima_la_plata_de_la_primera_división_de_argentina', 'NP00000', '0'),
  ('.', 'Fp', '0')],
 [('trayectoria', 'NP00000', '06709272'), ('.', 'Fp', '0')],
 [('debutar', 'VMIS3S0', '01176337'),
  ('en', 'SPS00', '0'),
  ('1', 'AO0FS0', '02098880'),
  ('el', 'DA0MS0', '0'),
  ('frente_a', 'SPS00', '0'),
  ('river_plate', 'NP00000', '0'),
  ('entra

In [43]:
from collections import defaultdict

In [77]:
dicc_backward = defaultdict(list)
dicc_forward = defaultdict(list)
for s in check:
    for i,w in enumerate(s):
        if i != 0:
            dicc_backward[w[0]].append((s[i-1][0], s[i-1][1]))
        else:
            dicc_backward[w[0]].append(('[W.Start]', '[T.Start]'))
        if i != len(s)-1:
            dicc_forward[w[0]].append((s[i+1][0], s[i+1][1]))
        else:
            dicc_forward[w[0]].append(('[W.End]', '[T.End]'))

In [78]:
dicc_backward['entrar']

[('river_plate', 'NP00000'), ('el', 'DA0MS0')]

In [79]:
dicc_forward['entrar']

[('desde', 'SPS00'), ('a', 'SPS00')]

In [138]:
dicc = dict()
vec = dict()
for fst,snd,trd in words:
    features = defaultdict(int)
    pos = 'PoS__' + snd
    if fst not in dicc and fst not in {'.', ',', ':', ';'} and fst.isalpha() and count[fst] > 1 and trd != 0:
        dicc[fst] = features
        vec[fst] = trd
        features[pos] = 1
        for i in dicc_backward[fst]:
            features[i[0]+"-1"] += 1
            features[i[1]+"-1"] += 1
        for i in dicc_forward[fst]:
            features[i[0]+"+1"] += 1
            features[i[1]+"+1"] += 1
        
    elif fst in dicc:
        if trd not in vec[fst]:
            dicc[fst] = features
            vec[fst] = trd
            for i in dicc_backward[fst]:
                features[i[0]+"-1"] += 1
                features[i[1]+"-1"] += 1
            for i in dicc_forward[fst]:
                features[i[0]+"+1"] += 1
                features[i[1]+"+1"] += 1
        has_it = dicc[fst]
        if pos in has_it:
            has_it[pos] += 1
        else:
            has_it[pos] = 1

In [139]:
dicc['con']

defaultdict(int,
            {'1-1': 1,
             '35+1': 1,
             'AQ0CS0-1': 3,
             'AQ0FP0-1': 1,
             'AQ0FS0-1': 3,
             'AQ0MP0-1': 1,
             'DA0FS0+1': 6,
             'DA0MP0+1': 2,
             'DA0MS0+1': 3,
             'DA0NS0+1': 1,
             'DI0FS0+1': 3,
             'DI0MP0+1': 1,
             'DI0MS0+1': 2,
             'DP3CS0+1': 3,
             'NCFP000+1': 2,
             'NCFP000-1': 2,
             'NCFS000+1': 1,
             'NCFS000-1': 6,
             'NCMN000+1': 1,
             'NCMP000+1': 3,
             'NCMP000-1': 1,
             'NCMS000+1': 3,
             'NCMS000-1': 5,
             'NP00000+1': 8,
             'NP00000-1': 6,
             'PR0CS000+1': 1,
             'PoS__SPS00': 41,
             'RG-1': 1,
             'VMII1S0-1': 1,
             'VMIP1S0-1': 1,
             'VMIP3P0-1': 1,
             'VMIP3S0-1': 5,
             'VMN0000-1': 1,
             'VMP00SF-1': 1,
             'VSIP3S0-

In [49]:
vec

{'a': '0',
 'altura': '03998008',
 'argentino': '00278090',
 'como': '0',
 'con': '0',
 'cousas': '0',
 'crear': '01111638',
 'cual': '0',
 'cuyo': '0',
 'dar': '00624932',
 'de': '0',
 'el': '0',
 'en': '0',
 'endofarticle': '0',
 'entre': '0',
 'equipo': '06093198',
 'estar': '01811792',
 'este': '0',
 'estilo': '03856995',
 'forma': '03856995',
 'formar': '01788486',
 'futbolista': '0',
 'gimnasia': '00275488',
 'haber': '01811792',
 'habitante': '06949059',
 'hacer': '01130277',
 'inglés': '05209017',
 'ir': '01371248',
 'jugar': '00727813',
 'le': '0',
 'localidad': '06368526',
 'minuto': '10943650',
 'mismo': '01990350',
 'o': '0',
 'para': '0',
 'partido': '05554348',
 'planta': '03138429',
 'poder': '01857455',
 'por': '0',
 'píldora': '03123093',
 'que': '0',
 'relato': '05392977',
 'se': '0',
 'ser': '01775973',
 'siempre': '0',
 'sinónimo': '04756968',
 'situar': '01354457',
 'solo': '02144240',
 'su': '0',
 'tableta': '03123093',
 'también': '0',
 'tener': '01508689',
 'the

In [122]:
matrix = []
vector = []
final_check = []
for key, value in dicc.items():    #preparo la lista de dicts y la lista de chequeo para recuperar el nombre al final
    final_check.append(key)
    matrix.append(value)
for key, value in vec.items():       #vector de synsets
    vector.append(int(value))

In [51]:
vector

[0,
 1775973,
 0,
 0,
 278090,
 727813,
 0,
 0,
 0,
 6093198,
 275488,
 0,
 0,
 10943650,
 10850147,
 5554348,
 0,
 0,
 624932,
 0,
 0,
 0,
 0,
 0,
 1788486,
 6368526,
 1990350,
 1354457,
 0,
 6949059,
 1811792,
 3998008,
 0,
 0,
 0,
 2144240,
 0,
 1508689,
 3138429,
 0,
 5392977,
 1111638,
 0,
 1857455,
 0,
 0,
 1371248,
 1130277,
 3856995,
 3856995,
 1811792,
 3123093,
 3123093,
 5209017,
 4756968,
 0]

In [123]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
X = v.fit_transform(matrix)

In [124]:
mejor_k = X.shape

In [125]:
k_best = mejor_k[1]*0.7

In [126]:
kbst

1681.3999999999999

In [127]:
##########################################Supervisada, clase: synset ##################################################

In [128]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [129]:
Y = SelectKBest(chi2, k=int(k_best)).fit_transform(X, vector)

In [130]:
#Normalización
import sklearn
Y = sklearn.preprocessing.normalize(Y)

In [134]:
#Clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=15)
km.fit(Y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=15, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [135]:
labels = km.predict(Y)

In [136]:
clusters = defaultdict(set)
for i, label in enumerate(labels):
    clusters[label].add(final_check[i])
print(clusters)

defaultdict(<class 'set'>, {6: {'mi', 'mediocampista', 'sillería', 'español', 'duración', 'nada', 'varios', 'uno', 'niño', 'hacer', 'cápsula', 'este', 'tipo', 'su', 'pastilla', 'manera', 'intercomunicación', 'moderno', 'numeroso', 'el', 'directo', 'igual', 'otro', 'medicina', 'nadie', 'francia', 'radio', 'planta', 'ese'}, 0: {'plata', 'la', 'centralita', 'nightwish', 'américa', 'argentina', 'bibliografía', 'sloan', 'gimnasia', 'youtube', 'grecia', 'enlaces', 'wesley', 'bond', 'cross', 'wanted', 'mcavoy', 'islandia', 'holopainen', 'jökulsárlón', 'endofarticle', 'timur', 'mendoza', 'himeto', 'fox', 'peramola', 'españa', 'referencias', 'castelao', 'praga', 'höfn', 'ecu', 'descripción'}, 3: {'ser', 'aunque', 'todo', 'médico', 'estadounidense', 'producir', 'escribir', 'formar', 'nacer', 'seguir', 'lanzar', 'resaltar', 'confirmar', 'él', 'deber', 'pero', 'o', 'ejecutar', 'dar', 'no', 'incluir', 'atravesar', 'estrenar', 'pasar', 'y', 'dejar'}, 14: {'habitante', 'activo', 'central', 'nuevo', '